# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
Use the BeautifulSoup package to transform the data in the table on the Wikipedia page into the above pandas dataframe

#### Install BeautifullSoup4

In [ ]:
!conda install beautifulsoup4 

#### Import Libraries

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Request and get data from wikipedia

In [18]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, "html.parser")
table = soup.find_all('table', {'class':'wikitable sortable'})

## 1. Create Data Frame

#### Set Data Frame

In [19]:
columns_name = ['Postcode', 'Borough', 'Neighborhood']
Postcode = pd.DataFrame(columns = columns_name)

#### Covert Data from wikipedia to data frame

In [20]:
A = table[0].find_all('tr')
for i in range(0,len(A)-1):
    B = A[i+1].find_all('td')
    postcode = B[0].text
    borough = B[1].text
    neighborhood = B[2].text   
    Postcode = Postcode.append({'Postcode': postcode,
                                'Borough': borough,
                                'Neighborhood':neighborhood.replace('\n','') }, ignore_index = True)


#### Clean Data

In [21]:
#  1. Only process the cells that have an assigned borough.
Postcode = Postcode[Postcode.Borough != 'Not assigned']

In [22]:
# 2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Postcode.loc[Postcode.Neighborhood == 'Not assigned', 'Neighborhood'] = Postcode.Borough

In [23]:
# 3. Group Neghborhood by Postcode
NewPostCode = Postcode.groupby(['Postcode','Borough'])['Neighborhood'].agg(lambda x: ','.join(set(x))).reset_index() 

#### Data Frame 

In [24]:
NewPostCode.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge"
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


## 2. Data Frame with latitude and the longitude coordinates

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. 

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

#### Load Geospatial_data

In [30]:
GeoDF = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
print('Data downloaded and read into a dataframe!')
GeoDF.head(10)

Data downloaded and read into a dataframe!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


#### Rename Postal Code to Postcode  

In [31]:
GeoDF.rename(columns ={'Postal Code':'Postcode'}, inplace = True) 
GeoDF.head(10)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


#### Add Latitude, Longtitude to NewPostCode
Merge NewPostCode and GeoDF

In [33]:
NewPostCode.reset_index(drop=True, inplace=True)
GeoDF.reset_index(drop=True, inplace=True)
NewPostCode = NewPostCode.astype(str)
GeoDF = GeoDF.astype(str)
NewPCGeo = pd.merge(NewPostCode, GeoDF, on = 'Postcode')
NewPCGeo.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686299999996,-79.19435340000001
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845351,-79.16049709999999
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.21691740000001
4,M1H,Scarborough,Cedarbrae,43.773136,-79.23947609999999
5,M1J,Scarborough,Scarborough Village,43.7447342,-79.23947609999999
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.7279292,-79.26202940000002
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711111700000004,-79.2845772
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.23947609999999
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657000000004,-79.2648481


#### Change decimal in Latitude and Longtitude to 6 decimal

In [34]:
NewPCGeo['Latitude'] = pd.to_numeric(NewPCGeo['Latitude'], errors='coerce')
NewPCGeo['Longitude'] = pd.to_numeric(NewPCGeo['Longitude'], errors='coerce') 

#### Data Frame with Latitude and Longitude

In [36]:
NewPCGeo

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848
